In [ ]:
# import all dependencies

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
# Open an Edge browser window, maximize it, and navigate to the Cars24 used car listing page with filters

driver=webdriver.Edge()
driver.maximize_window()
driver.get("https://www.cars24.com/buy-used-car?f=listingPrice%3Abw%3A150000%2C600000&sort=bestmatch&serveWarrantyCount=true&gaId=1200561372.1721151183&listingSource=TabFilter&storeCityId=2378")
for k in range(20):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)


In [ ]:
# Parse the current HTML content of the WebDriver instance into a BeautifulSoup object

soup=BeautifulSoup(driver.page_source,'html.parser')
print(soup)

In [ ]:
# Find all elements with the particular class name

result=soup.find_all("div",{"class":"_2ujGx"}) 
result

In [ ]:
# Find all anchor tags with the particular class

anchor_tag=soup.find_all("a",{"class":"IIJDn"}) 
anchor_tag

In [ ]:
# Extract href (link) values from the anchor tags and store them in a list

a_list=[]
for a_tag in anchor_tag:
    href_value = a_tag.get('href')
    a_list.append(href_value)

In [ ]:
def scrape_data_from_link(link):
    driver = webdriver.Edge()
    driver.maximize_window()
    driver.get(link)
    time.sleep(5)
    
    # Use dynamic waiting techniques instead of a fixed sleep
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Initialised an empty dictionary
    
    data = {"brand": [],
            "rege":[],
            "payment":[],
            "location":[]
            }
   # Find HTML elements based on class names
    
    brand_tag = soup.find("h1", {"class": "_2Ximl"})        
    reg_no=soup.find("div", {"class": "_36EKv"})
    Pay=soup.find("div", {"class": "_2j-eI"})
    loca=soup.find("li", {"class": "_1Rvdw"})
    
   # Extract and clean text content
    
    brand = brand_tag.text.strip()
    registration = reg_no.text.strip()
    payment = Pay.text.strip()
    location = loca.text.strip()

    # Append data in the dictionary
    
    data["brand"].append(brand)
    data["rege"].append(registration)
    data["payment"].append(payment)
    data["location"].append(location)    
    
    driver.quit()
    return data
    
# Looping throught each link and storing the scraped data in an empty dictionary

scraped_data = {}
for link in a_list:
    data = scrape_data_from_link(link)
    scraped_data[link] = data


In [ ]:
# Create a dataframe out of scraped data 

df=pd.DataFrame(scraped_data)

In [ ]:
# Transpose the DataFrame (swap rows and columns)

df=df.T

In [ ]:
# Print a concise summary of the DataFrame

df.info()

In [ ]:
# Save the dataframe as csv file

df.to_csv("uncleaned_cars24_data.csv")